Prepare list of paths for pdf files

In [3]:
import os

# path to folder with pdf files
folder_path = ""
pdfsList = os.listdir(folder_path)

full_paths = []
for e in pdfsList:
    full_path = os.path.join(folder_path, e)
    full_paths.append(full_path)

Read all pdfs pages to array

In [4]:
import os
import PyPDF2

all_pages_content = []

for e in full_paths:
    if e.endswith(".pdf"):
        with open(e, "rb") as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            filename = os.path.basename(e)
            for i, page in enumerate(reader.pages):
                page_content = page.extract_text()
                all_pages_content.append({
                    "filename": filename,
                    "page_number": i + 1,
                    "content": page_content
                })



replace all enters with " "

In [7]:
import re

split_content = []

for page in all_pages_content:
    content = page.get("content")
    if content:
        text = content.replace("\n", " ").strip()

        sentences = re.split(r'(?<=[.?!])\s+(?=[A-ZŁŻŚĆŃÓ])', text)

        # Dodaj każde zdanie jako osobny chunk
        for sentence in sentences:
            cleaned = sentence.strip()
            if cleaned:  # pomiń puste
                split_content.append({
                    "filename": page["filename"],
                    "page_number": page["page_number"],
                    "content": cleaned
                })

all_pages_content = split_content

print(all_pages_content)


[{'filename': 'wykład 11.pdf', 'page_number': 1, 'content': 'Microsoft ConfidentialPRAWNE PODSTAWY DZIAŁALNOŚCI GOSPODARCZEJ  WYKŁAD XI Halszka Suszek -Borowska'}, {'filename': 'wykład 11.pdf', 'page_number': 2, 'content': 'ZAGADNIENIE: WSZYSTKIE PRAWA (AUTORSKIE) ZASTRZEŻONE?'}, {'filename': 'wykład 11.pdf', 'page_number': 2, 'content': 'Każdego dnia mamy do czynienia z twórczymi produktami intelektu.'}, {'filename': 'wykład 11.pdf', 'page_number': 2, 'content': 'Artykuły, książki, programy komputerowe,  muzyka, grafika… Celem prawa autorskiego jest ochrona wszystkich tych utworów.'}, {'filename': 'wykład 11.pdf', 'page_number': 2, 'content': 'To co tworzymy i to co prawo  autorskie chroni, nazywamy własnością intelektualną.'}, {'filename': 'wykład 11.pdf', 'page_number': 2, 'content': 'Podczas tego wykładu, dowiesz się kim jest autor i jakie  posiada prawa.'}, {'filename': 'wykład 11.pdf', 'page_number': 2, 'content': 'Poznasz także inne podstawowe zagadnienia związane z prawem autor

In [8]:
import faiss
import numpy as np
import json
from sentence_transformers import SentenceTransformer

INDEX_FILE = "PPB.index"
META_FILE = "PPB_metadata.json"


def build_and_save_index(chunks, model):
    print("[✔] Tworzenie embeddingów i zapisywanie...")

    embeddings = [model.encode(chunk["content"]) for chunk in chunks]
    dim = len(embeddings[0])
    index = faiss.IndexFlatL2(dim)
    index.add(np.array(embeddings, dtype="float32"))

    faiss.write_index(index, INDEX_FILE)
    with open(META_FILE, "w", encoding="utf-8") as f:
        json.dump(chunks, f, ensure_ascii=False, indent=2)
    print(f"[✔] Zapisano indeks do '{INDEX_FILE}' i metadane do '{META_FILE}'.")




build_and_save_index(all_pages_content, SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2'))


[✔] Tworzenie embeddingów i zapisywanie...
[✔] Zapisano indeks do 'PPB.index' i metadane do 'PPB_metadata.json'.


In [1]:
print(all_pages_content)

NameError: name 'all_pages_content' is not defined